The following notebook shows how to combine **Semantic Kernel and AutoGen**, leveraging
SK's ability to work with **safe plugins** and creating **planners** to achieve
users' goals, and AutoGen's **conversational programming** approach to iteratively
complete a task.

Several AutoGen examples focus on code generation, while in this case we take a
safer approach, **controlling what our Agents can do, providing a list of Plugins**,
and allowing AI to execute only the plugins loaded into Semantic Kernel.

The scenario is pretty simple and easy to extend, we provide two Plugins:

1. A Plugin to search the Web, using Bing
2. A Plugin to parse web pages

and ask Semantic Kernel to answer a question, if possible.

Feel free to add more plugins, to augment the agent and increase the scope of work
the agent can do.

The following block installs the required library in the current Jupyter kernel.
We recommend using at least Python 3.10, though 3.8+ should work fine too.

In [ ]:
# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install --upgrade semantic-kernel==0.3.14.dev0
!{sys.executable} -m pip install --upgrade pyautogen==0.1.14
!{sys.executable} -m pip install --upgrade beautifulsoup4~=4.12

Before proceeding please provide some API keys.

To get a Bing API Key, you should deploy a Bing Search resource on Azure,
then look for the credentials under "Keys and Endpoints" in the Azure Portal.

Semantic Kernel and AutoGen leverage **OpenAI Function Calling** (version 0613).
Select here whether you'd like to use Azure OpenAI or OpenAI (see `type`), and
provide the usual LLM credentials.

For OpenAI the code automatically uses GPT 3.5 turbo. For Azure OpenAI, please
provide a deployment name pointing either to GPT 3.5 turbo or GPT4 version 0613.

In [ ]:
# Configure your credentials here

bing_api_key = "..." # Replace with your Bing API key

llm_config = {
    "type"            : "azure",  # "azure" or "openai"

    "openai_api_key"  : "sk-...", # OpenAI API Key

    "azure_deployment": "",       # Azure OpenAI deployment name
    "azure_api_key"   : "",       # Azure OpenAI API key in the Azure portal
    "azure_endpoint"  : ""        # Endpoint URL for Azure OpenAI, e.g. https://contoso.openai.azure.com/
}

The following block is very straightforward: load Semantic Kernel :-)

In [ ]:
# Load Semantic Kernel
import semantic_kernel

kernel = semantic_kernel.Kernel()

print("Kernel loaded.")

Here's is where we control how our agents will get work done. We load some plugins
into Semantic Kernel. 

You can find the plugins under the `plugins` folder, and you can also add more
plugins to augment the agent.

In [ ]:
# Load Plugins
from plugins.sk_bing_plugin import BingPlugin
from plugins.sk_web_pages_plugin import WebPagesPlugin

kernel.import_skill(BingPlugin(bing_api_key))
kernel.import_skill(WebPagesPlugin())

print("Plugins ready.")

Finally the last step to instantiate our new AutoGen Planner, integrating AutoGen
with Semantic Kernel, and leveraging the well know Planner pattern, now with
conversational programming.

In [ ]:
# Load SK AutoGen Planner
from planning.autogen_planner import AutoGenPlanner

sk_planner = AutoGenPlanner(kernel, llm_config)

assistant = sk_planner.create_assistant_agent("Assistant")
worker = sk_planner.create_user_agent("Worker", max_auto_reply=4, human_input="NEVER")

print("Planner ready.")

Lastly: define a goal and start the agents!

In [ ]:
# Run!

question = "what's the current Microsoft stock price?"

worker.initiate_chat(assistant, message=question)

The final result will look something like this, showing our agents working
through the given task and eventually showing the result:

![images/12-1.png](images/12-1.png)